<a href="https://colab.research.google.com/github/vanyalzr/spike_data_generation/blob/master/generate_spike_raster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! apt install build-essential git python3-dev python3-setuptools python3-scipy python3-matplotlib python3-sympy cython3 > /dev/null

In [0]:
! pip install ANNarchy > /dev/null

In [0]:
! rm -rf ./spike_data_generation && git clone https://github.com/vanyalzr/spike_data_generation.git

In [0]:
import matplotlib.pyplot as plt

from spike_data_generation.models import SpikingNet
from spike_data_generation.utils import process_raster
from spike_data_generation.mpladeq import beautify_mpl, prettify

beautify_mpl()

In [0]:
regular_spiking_params = {'a': 0.02, 'b': 0.2, 'c': -65.0, 'd': 8.0, 'v': -60}

rs_net = SpikingNet(n_neurons=100,
                    parameters=regular_spiking_params,
                    noise=3.9,
                    e_neurons=1,
                    i_neurons=1,
                    e_weight_max=0,
                    i_weight_max=0)


rs_times, rs_neurons, rate = rs_net.run(time=1e3)
_, rs_isis, rs_isi = process_raster(rs_times, rs_neurons)

In [0]:
plt.scatter(rs_times, rs_neurons, s=50., marker='|')
plt.xlabel('Time, ms')
plt.ylabel('Neuron #')
prettify(figsize=(20, 10))   # rerun the cell twice to get desired figsize